In [ ]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet
from sklearn.model_selection import train_test_split

# Check if Cuda is Available

In [ ]:
import torch

print(f'{torch.cuda.is_available() = }')
print(f'{torch.cuda.device_count() = }')

In [ ]:
!pip install ultralytics

In [ ]:
!pip install -U ipywidgets

# Setting up the dataset paths

In [ ]:
dataset_path = '/kaggle/input/car-plate-detection'

In [ ]:
# extract first sequence of digit from string and return it as an integer.
# and if no then return 0
import re

def number_from_string(filename):
    
    match = re.search(r'(\d+)', filename)
    if match:
        return int(match.group(0))
    else:
        return 0

# example
print(number_from_string('fidle1234.txt'))
print(number_from_string('no_number_here'))

In [ ]:
# dictionary to store labels and image informations
label_dict = dict(
    img_path=[],
    xmin = [],
    xmax = [],
    ymin = [],
    ymax = [],
    img_w= [],
    img_h= []
)

# get the list of xml files from annotation directory
xml_files = glob(f'{dataset_path}/annotations/*.xml')

# process each xml file, sorted by the numerical value in the filename

i=0
for filename in sorted(xml_files, key=number_from_string):
    
    # parse the XML file
    info = xet.parse(filename)
    root = info.getroot()
    
    # find the object element in the XML and extract bounding box information
    member_obj = root.find('object')
    labels_info= member_obj.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)
    
    # Get the image filename and construct the full path to the image
    img_name =root.find('filename').text 
    img_path = os.path.join(dataset_path, 'images', img_name)
    
    # Append the extracted information to the respective fields in label dict
    label_dict['img_path'].append(img_path)
    label_dict['xmin'].append(xmin)
    label_dict['xmax'].append(xmax)
    label_dict['ymin'].append(ymin)
    label_dict['ymax'].append(ymax)
    
    height, width, _ = cv2.imread(img_path).shape
    label_dict['img_w'].append(width)
    label_dict['img_h'].append(height)

alldata = pd.DataFrame(label_dict)
alldata.head()

In [ ]:
train, test = train_test_split(alldata, test_size=0.1, random_state=42)
train, val  = train_test_split(train, train_size=8/9 , random_state=42)

print(f'''
      len(train) = {len(train)}
      len(val)   = {len(val)}
      len(test)  = {len(test)}
''')

In [ ]:
def folders_in_YOLO_format(split_name, split_df):
    
    # folder structure for a dataset in YOLO format
    labels_path = os.path.join('datasets', 'car_license_plate_new',split_name, 'labels')
    images_path = os.path.join('datasets', 'car_license_plate_new',split_name, 'images')
    
    # create directories for labels and images
    os.makedirs(labels_path)
    os.makedirs(images_path)
    
    # iterate over each row in data format
    for _, row in split_df.iterrows():
        img_name, img_extension = os.path.splitext(os.path.basename(row['img_path']))
        
        # calculating bounding boxes in YOLO formate
        x_center = (row['xmin']+row['xmax'])/2/row['img_w']
        y_center = (row['ymin']+row['ymax'])/2/row['img_h']
        width = (row['xmax']-row['xmin'])/row['img_w']
        height= (row['ymax']-row['ymin'])/row['img_h']
        
        # save the label in YOLO formate
        label_path = os.path.join(labels_path, f'{img_name}.txt')
        # put the coordinate of bounding boxes in label path
        with open(label_path, 'w') as file:
            file.write(f"0 {x_center:.4f} {y_center:.4f} {width:.4f} {height:.4f}\n")
        
        # copy the images to the image directory
        shutil.copy(row['img_path'], os.path.join(images_path, img_name+img_extension))
    
    print(f"Created '{images_path}' and '{labels_path}'")  

In [ ]:
# create folders in YOLO format for train, test and validation data
folders_in_YOLO_format('train', train)
folders_in_YOLO_format('val', val)
folders_in_YOLO_format('test', test)

In [ ]:
os.getcwd()

In [ ]:
# Directory Paths
image_dir = 'datasets/car_license_plate_new/train/images'
label_dir = 'datasets/car_license_plate_new/train/labels'

#  Get the first images files
image_files = sorted(os.listdir(image_dir))
first_img_files = image_files[0]

# construct path for the 1st image and labels
image_path = os.path.join(image_dir, first_img_files)
label_path = os.path.join(label_dir, os.path.splitext(first_img_files)[0]+'.txt')

# load the image and change color in rgb format
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.axis('off')

In [ ]:
# from label path get the coordinate of bounding box
with open(label_path , 'r') as f:
    lines = f.readlines()

for line in lines:
    class_id, x_center, y_center, width, height = map(float,line.strip().split())
    img_height, img_width, _ = image.shape
    
    # convert YOLO format to bounding box format
    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height*= img_height
    
    # calculate the top-left and bottom-right coordinates of the bounding box
    x1 = int(x_center-width/2)
    y1 = int(y_center-height/2)
    x2 = int(x_center+width/2)
    y2 = int(y_center+height/2)
    
    # draw the bounding box on the images 
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 250, 0), 2)
    
# display the image with bounding box using matplotlib
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
# Defin the content of the dataset.yaml file
datasets_yaml = '''
path: car_license_plate_new

train: train/images
val: val/images
test: test/images

nc: 1
names: ['license_plate']

'''

with open('datasets.yaml', 'w') as file:
    file.write(datasets_yaml)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

In [ ]:
model.train(
    data = 'datasets.yaml',
    epochs=100,
    batch=16, 
    device='cuda',
    imgsz=320,
    cache=True
)

In [ ]:
# find the most recent training log directory
log_dir = max(glob('runs/detect/train*'), key=number_from_string)


# load the training result from the csv file
result = pd.read_csv(os.path.join(log_dir, 'results.csv'))
result.columns = result.columns.str.strip()

In [ ]:
result

In [ ]:
result.columns

In [ ]:
epochs = result.index+1 # epochs are zero idx so, we add 1

# mean avg precision are IOU=0.5 and 0.5:0.95
map_0_5 = result['metrics/mAP50(B)'] 
map_0_5_0_95 = result['metrics/mAP50-95(B)']

plt.figure(figsize=(10, 8))
plt.plot(epochs, map_0_5, label='map@0.5')
plt.plot(epochs, map_0_5_0_95, label='map@0.5:0.95')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy v/s Epochs')
plt.legend()
plt.grid()
plt.show()

In [ ]:
model.save('license_plate_model.pt')

# Prediction

In [ ]:
result = model.predict('/kaggle/working/datasets/car_license_plate_new/test/images/Cars154.png')

In [ ]:
result

In [ ]:
os.listdir('runs/detect')

In [ ]:
for res in result:
    for box in res.boxes:
        print(box)
        break
    break

In [ ]:
def predict_and_plot(path_test_car):
    results = model.predict(path_test_car)
    image = cv2.imread(path_test_car)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            cv2.rectangle(image, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(image, f'{confidence*100:.2f}%', (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [ ]:
predict_and_plot('/kaggle/working/datasets/car_license_plate_new/test/images/Cars39.png')

# **OCR(OPTICAL CHARACTER RECOGNITION)**

In [ ]:
!pip install pytesseract

In [ ]:
import pytesseract
from pytesseract import Output

In [ ]:
def predict_and_plot(path_test_car):
    results = model.predict(path_test_car)
    image = cv2.imread(path_test_car)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            cv2.rectangle(image, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(image, f'{confidence*100:.2f}%', (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
            # crop the bounding box from the image for OCR
            roi = image[y1:y2, x1:x2]
            
            # perform OCR on the cropped image
            text = pytesseract.image_to_string(roi, config='--psm 6')
            print(f"Detected Text: {text}")
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [ ]:
predict_and_plot('/kaggle/working/datasets/car_license_plate_new/test/images/Cars39.png')